In [4]:
import numpy as np
from pyproj import Proj, transform, CRS, Transformer, Geod
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

# biblioteka pyproj -> PL2000, PL92, UTM, LAEA, LCC

# pio.renderers.default = 'browser'

degree_sign = u"\N{DEGREE SIGN}"
# Radiany na stopnie, minuty, sekundy
def rad2dms(rad):
    dd = np.rad2deg(rad)
    dd = dd
    deg = int(np.trunc(dd))
    mnt = int(np.trunc((dd-deg) * 60))
    sec = ((dd-deg) * 60 - mnt) * 60
    mnt = abs(mnt)
    sec = abs(sec)
    if sec > 59.99999:
        sec = 0
        mnt += 1
    if sec < 10:
        sec = f"0{sec:.5f}"
    else:
        sec = f"{sec:.5f}"
    if mnt < 10:
        mnt = f"0{mnt}"
    dms = (f"{deg}{degree_sign} {mnt}' {sec}''")
    return dms

# Stopnie dziesiętne na stopnie, minuty, sekundy
def deg2dms(dd):
    deg = int(np.trunc(dd))
    mnt = int(np.trunc((dd-deg) * 60))
    sec = ((dd-deg) * 60 - mnt) * 60
    mnt = abs(mnt)
    sec = abs(sec)
    if sec < 10:
        sec = f"0{sec:.5f}"
    else:
        sec = f"{sec:.5f}"
    if mnt < 10:
        mnt = f"0{mnt}"
    dms = (f"{deg}{degree_sign}{mnt}'{sec}''")
    return dms

# Kody układów współrzędnych
input_code = 4326
output_names = ['PL-2000', 'PL-1992', 'UTM', 'LAEA', 'LCC']
output_codes = [2176, 2180, 32633, 3035, 3034]

input_proj = CRS.from_epsg(4326)
proj_2000 = CRS.from_epsg(2176)
proj_92 = CRS.from_epsg(2180)
proj_utm = CRS.from_epsg(32633)
proj_laea = CRS.from_epsg(3035)
proj_lcc = CRS.from_epsg(3034)

output_projections = [proj_2000, proj_92, proj_utm, proj_laea, proj_lcc]

# Parametry z ćwiczenia 3
lines = ['1 - 2', '2 - 3', '3 - 4', '4 - 1']
lengths_given = [40000.000, 100000.000, 40000.000, 100000.000]
azimuths_given = [0, 90, 180, 270]
azimuths_given_dms = [deg2dms(azimuths_given[i]) for i in range(4)]

azimuths_kivioj = [0, 91.23842788254149, 180, 268.77819133933843]
azimuths_kivioj_dms = [deg2dms(azimuths_kivioj[i]) for i in range(4)]

azimuths_rect = [0, 91.23842788254149, 180, -90.00247250201006]
azimuths_rect_dms = [deg2dms(azimuths_rect[i]) for i in range(4)]

azimuths_vincent = [0, 90.00000000009787, 180, 271.23030946924604]
azimuths_vincent_dms = [deg2dms(azimuths_vincent[i]) for i in range(4)]
azimuths_back_vincent = [180, 271.2384278825927, 360, 89.99752749815782]
azimuths_back_vincent_dms = [deg2dms(azimuths_back_vincent[i]) for i in range(4)]
lengths_vincent = [40000.00000, 100000.00001, 40000.00000, 100862.55117]

areas_ex3 = [4016880873.853, 4016.881]

# Współrzędne punktów
phis = [53.75, 54.10937712005116, 54.099667067097876, 53.74028936233757] 
lambdas = [15.25, 15.25, 16.778726126645378, 16.778726126645378]
lamb0 = np.radians(15)

x1_out, x2_out, x3_out, x4_out = [], [], [], []
y1_out, y2_out, y3_out, y4_out = [], [], [], []

x_out = [x1_out, x2_out, x3_out, x4_out]
y_out = [y1_out, y2_out, y3_out, y4_out]

# Pętla transformacji
for proj in output_codes:
    for i in range(4):
        transformer = Transformer.from_crs(input_code, proj)
        y, x = transformer.transform(phis[i], lambdas[i])
        x_out[i].append(x)
        y_out[i].append(y)

P2176 = 0
P2180 = 0
P32633 = 0
P3035 = 0
P3034 = 0

areas_epsg = [P2176, P2180, P32633, P3035, P3034]
P2176_con = 0
P2180_con = 0
P32633_con = 0
P3035_con = 0
P3034_con = 0
areas_con_epsg = [P2176_con, P2180_con, P32633_con, P3035_con, P3034_con]

# Pętla pola powierzchni
for u in range(5):
    for i in range(4):
        # kij
        j = (i + 1) % 4 
        k = (i - 1) % 4
        # Gauss
        area = x_out[i][u] * (y_out[j][u] - y_out[k][u])
        area_con = y_out[i][u] * (x_out[j][u] - x_out[k][u])
        areas_epsg[u] += area
        areas_con_epsg[u] += area_con

areas_epsg = [round(abs(areas_epsg[i] / 2), 3) for i in range(5)]
areas_con_epsg = [round(abs(areas_con_epsg[i] / 2), 3) for i in range(5)]
areas_epsg_km = [round(areas_epsg[i] / 1000000, 3) for i in range(5)]

# Pole na elipsoidzie
phis_poly = [phis[0], phis[1], phis[2], phis[3], phis[0]]
lambdas_poly = [lambdas[0], lambdas[1], lambdas[2], lambdas[3], lambdas[0]]

area4326, perimeter4326 = Geod(ellps='WGS84').geometry_area_perimeter(
    Polygon(
        LineString([
            Point(lambdas_poly[i], phis_poly[i]) for i in range(5)])
        )
    )
area4326 = round(abs(area4326), 3)

# Parametry elipsoidy 
a = 6378137.0 
a2 = a ** 2
e2 = 0.00669438002290
b2 = a2 * (1 - e2)
e22 = (a2- b2)  / b2

# Współczynnik zniekształcenia
m2000 = 0.999923
m1992 = 0.9993

# Promienie krzywizny
def M_and_N(phi):
    sin_phi = np.sin(phi)
    M = a * (1 - e2) / (1 - e2 * sin_phi**2)**(3/2)
    N = a / np.sqrt(1 - e2 * sin_phi**2)
    return M, N

lengths_2000 = []
lengths_1992 = []
lengths_gk2000 = []
lengths_gk1992 = []
R2000 = []
R1992 = []
diff2000 =[]
diff1992 = []
lengths_elip2000 = []
lengths_elip1992 = []
lengths_table = [lengths_2000, lengths_gk2000, lengths_elip2000, lengths_1992, lengths_gk1992, lengths_elip1992]

azimuths_2000 = []
azimuths_back_2000 = []
azimuths_1992 = []
azimuths_back_1992 = []
azimuths_table = [azimuths_2000, azimuths_back_2000, azimuths_1992, azimuths_back_1992]

P_1_2000 = 0
P_2_2000 = 0
P_1_1992 = 0
P_2_1992 = 0
areas_table = [P_1_2000, P_2_2000, P_1_1992, P_2_1992]

phis = np.deg2rad(phis)
lambdas = np.deg2rad(lambdas)

# Redukcje
for i in range(4):
    # kij
    j = (i + 1) % 4
    k = (i - 1) % 4

    # Redukcja długości 
    phi_m = (phis[i] + phis[j]) / 2
    M, N = M_and_N(phi_m)
    Rm = np.sqrt(M * N)

    # Długość odcinka na płaszczyźnie
    length_2000 = np.sqrt((x_out[i][0] - x_out[j][0])**2 + (y_out[i][0] - y_out[j][0])**2)
    length_1992 = np.sqrt((x_out[i][1] - x_out[j][1])**2 + (y_out[i][1] - y_out[j][1])**2)
    lengths_2000.append(round(length_2000, 3))
    lengths_1992.append(round(length_1992, 3))

    # Długość odcinka na płaśzczyźnie Gaussa-Krugera
    length_gk2000 = length_2000 / m2000
    length_gk1992 = length_1992 / m1992
    lengths_gk2000.append(round(length_gk2000, 3))
    lengths_gk1992.append(round(length_gk1992, 3))

    # Redukcja długości
    r2000 = length_gk2000 /10000* (y_out[i][0] ** 2 + y_out[i][0] * y_out[j][0] + y_out[j][0] ** 2) / (6 * Rm ** 2) 
    r1992 = length_gk1992 /10000 * (y_out[i][1] ** 2 + y_out[i][1] * y_out[j][1] + y_out[j][1] ** 2) / (6 * Rm ** 2)

    R2000.append(r2000)
    R1992.append(r1992)

    # Długość odcinka na elipsoidzie
    length_elip2000 = length_gk2000 - r2000
    length_elip1992 = length_gk1992 - r1992
    lengths_elip2000.append(round(length_elip2000, 3))
    lengths_elip1992.append(round(length_elip1992, 3))

    diff2000.append(lengths_vincent[i] - length_elip2000)
    diff1992.append(lengths_vincent[i] - length_elip1992)

    # Redukcja azymutów
    d_lambda = lambdas[i] - lamb0
    t = np.tan(phis[0])
    eta2 = e22 * np.cos(phis[i]) ** 2

    # CZY TA REDUKCJA JEST DOBRA?
    # t = np.tan(phis[0])
    # M, N = M_and_N(phis[0])

    for u in range(2):
        # Kąt kierunkowy
        delta_x = x_out[j][u] - x_out[i][u]
        delta_y = y_out[j][u] - y_out[i][u]
        alpha_ab = np.arctan2(delta_y, delta_x)
        delta_x = x_out[i][u] - x_out[j][u]
        delta_y = y_out[i][u] - y_out[j][u]
        alpha_ba = np.arctan2(delta_y, delta_x)
       
        alpha_ab = np.arctan((y_out[j][u] - y_out[i][u]) / (x_out[j][u] - x_out[i][u]))
        alpha_ba = np.arctan((y_out[i][u] - y_out[j][u]) / (x_out[i][u] - x_out[j][u]))

        # Zbieżność południków
        gamma_a = (d_lambda * np.sin(phis[i])) + ((d_lambda ** 3 / 3) * np.sin(phis[i]) * np.cos(phis[i]) ** 2 * (1 + 3 * eta2 + 2 * eta2 ** 2)) + ((d_lambda ** 5 / 15) * np.sin(phis[i]) * np.cos(phis[i]) ** 4 * (2 - t ** 2)) 
        gamma_b = (d_lambda * np.sin(phis[j])) + ((d_lambda ** 3 / 3) * np.sin(phis[j]) * np.cos(phis[j]) ** 2 * (1 + 3 * eta2 + 2 * eta2 ** 2)) + ((d_lambda ** 5 / 15) * np.sin(phis[j]) * np.cos(phis[j]) ** 4 * (2 - t ** 2))
        
        # Redukcja kierunków
        delta_ab = (x_out[j][u] - x_out[i][u]) * (2 * y_out[u][i] + y_out[u][j]) / (6 * Rm ** 2)
        delta_ba = (x_out[i][u] - x_out[j][u]) * (2 * y_out[u][j] +  y_out[u][i]) / (6 * Rm ** 2)

        # Azymut odcinka
        A_ab = alpha_ab + gamma_a + delta_ab
        A_ba = alpha_ba + gamma_b + delta_ba
        A_ab = np.degrees(A_ab)
        A_ba = np.degrees(A_ba)
        if A_ab < 0:
            A_ab += 360
        if A_ba < 0:
            A_ba += 360

        if u == 0:
            azimuths_2000.append(A_ab)
            azimuths_back_2000.append(A_ba)
        elif u == 1:
            azimuths_1992.append(A_ab)
            azimuths_back_1992.append(A_ba)

    # Pole powierzchni z Gaussa
    P_1_2000 += x_out[i][0] * (y_out[j][0] - y_out[k][0])
    P_2_2000 += y_out[i][0] * (x_out[j][0] - x_out[k][0])

    P_1_1992 += x_out[i][1] * (y_out[j][1] - y_out[k][1])
    P_2_1992 += y_out[i][1] * (x_out[j][1] - x_out[k][1])

    # gamma2000 = y_out[0][i] / N * t * (1 - y_out[0][i] / 3 * N ** 2 * (1 + t ** 2 - eta2 - 2 * eta2 ** 2) + y_out[0][i] ** 2 / 15 * N ** 4 * (2 + 5 * t ** 2 + 3 * t ** 4))


azimuths_2000_dms = [deg2dms(azimuths_2000[i]) for i in range(4)]
azimuths_back_2000_dms = [deg2dms(azimuths_back_2000[i]) for i in range(4)]

azimuths_1992_dms = [deg2dms(azimuths_1992[i]) for i in range(4)]
azimuths_back_1992_dms = [deg2dms(azimuths_back_1992[i]) for i in range(4)]

P2000_1 = [round((P_1_2000 / 2), 3), round(P_1_2000 / 2000000, 3)]
P2000_2 = [round((P_2_2000 / 2), 3), round(P_2_2000 / 2000000, 3)]
P1992_1 = [round((P_1_1992 / 2), 3), round(P_1_1992 / 2000000, 3)]
P1992_2 = [round((P_2_1992 / 2), 3), round(P_2_1992 / 2000000, 3)]


# Data frames for latex tables
nr = [1, 2, 3, 4]
# Coordinates
# PL-2000
df_2000 = pd.DataFrame({'nr': nr, 'x': [x1_out[0], x2_out[0], x3_out[0], x4_out[0]], 'y': [y1_out[0], y2_out[0], y3_out[0], y4_out[0]]}, index=nr)
print(df_2000.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|', header=True, position='!ht',
caption='Współrzędne punktów w układzie PL-2000', label='tab:2000'))
# PL-1992
df_1992 = pd.DataFrame({'nr': nr, 'x': [x1_out[1], x2_out[1], x3_out[1], x4_out[1]], 'y': [y1_out[1], y2_out[1], y3_out[1], y4_out[1]]}, index=nr)
print(df_1992.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|', header=True, position='!ht',
caption='Współrzędne punktów w układzie PL-1992', label='tab:1992'))
# UTM
df_utm = pd.DataFrame({'nr': nr, 'x': [x1_out[2], x2_out[2], x3_out[2], x4_out[2]], 'y': [y1_out[2], y2_out[2], y3_out[2], y4_out[2]]}, index=nr)
print(df_utm.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|', header=True, position='!ht',
caption='Współrzędne punktów w układzie UTM', label='tab:utm'))
# LAEA
df_laea = pd.DataFrame({'nr': nr, 'x': [x1_out[3], x2_out[3], x3_out[3], x4_out[3]], 'y': [y1_out[3], y2_out[3], y3_out[3], y4_out[3]]}, index=nr)
print(df_laea.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|', header=True, position='!ht',
caption='Współrzędne punktów w układzie LAEA', label='tab:laea'))
# LCC
df_lcc = pd.DataFrame({'nr': nr, 'x': [x1_out[4], x2_out[4], x3_out[4], x4_out[4]], 'y': [y1_out[4], y2_out[4], y3_out[4], y4_out[4]]}, index=nr)
print(df_lcc.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|', header=True, position='!ht',
caption='Współrzędne punktów w układzie LCC', label='tab:lcc'))

# Lengths
# PL-2000
df_dl2000 = pd.DataFrame({'nr': lines, 'Dł. dane': lengths_given, 'Dł.Vincenty': lengths_vincent, 'PL-2000': lengths_2000, 'Gauss-Krüger': lengths_gk2000, 'Elipsoida': lengths_elip2000}, index=nr)
print(df_dl2000.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|c|c|', header=True, position='!ht',
caption='Długości odcinków w układzie PL-2000', label='tab:dl2000'))
# PL-1992
df_dl1992 = pd.DataFrame({'nr': lines, 'Dł. dane': lengths_given, 'Dł.Vincenty': lengths_vincent,'PL-1992': lengths_1992, 'Gauss-Krüger': lengths_gk1992, 'Elipsoida': lengths_elip1992}, index=nr)
print(df_dl1992.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|c|c|', header=True, position='!ht',
caption='Długości odcinków w układzie PL-1992', label='tab:dl1992'))

df_dl = pd.DataFrame({'A-B': lines, 'Dł. dane': lengths_given, 'Dł.Vincenty': lengths_vincent, 'PL-2000': lengths_elip2000, 'PL-1992': lengths_elip1992})
print(df_dl.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|c|', header=True, position='!ht',
caption='Długości odcinków', label='tab:dl1992'))

# Azimuths
# PL-2000
df_az2000 = pd.DataFrame(
    {'nr': lines, 'Az. dany': azimuths_given_dms, 'Kivioj': azimuths_kivioj_dms, 'Vincenty': azimuths_back_vincent_dms, 'Vincenty (odwrotny)': azimuths_vincent_dms, 
    'PL-2000': azimuths_2000_dms, 'PL-2000 (odwrotny)': azimuths_back_2000_dms}, index=nr)
print(df_az2000.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|c|', header=True, position='!ht',
caption='Azymuty odcinków w układzie PL-2000', label='tab:az2000'))
# PL-1992
df_az1992 = pd.DataFrame({'nr': lines, 'Az. dany': azimuths_given_dms, 'Vincenty': azimuths_back_vincent_dms, 'Vincenty (odwrotny)': azimuths_vincent_dms, 
    'PL-1992': azimuths_1992_dms, 'PL-1992 (odwrotny)' : azimuths_back_1992_dms}, index=nr)
print(df_az1992.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|', header=True, position='!ht',
caption='Azymuty odcinków w układzie PL-1992', label='tab:az1992'))

# Areas
df_area = pd.DataFrame({'Ćw. 3': areas_ex3, 'PL-2000': P2000_1, 'PL-2000' : P2000_2, 'PL-1992': P1992_1, 'PL-1992': P1992_2})
print(df_area.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|c|c|', header=True, position='!ht',
caption='Pole powierzchni w układach PL-2000 i PL-1992', label='tab:area'))

df_areas = pd.DataFrame({'Układ': output_names, 'Pole': areas_epsg, 'Pole kontrola': areas_epsg_km}, index=range(5))
print(df_areas.to_latex(index=False, float_format="%.3f", column_format='|c|c|c|,', header=True, position='!ht',
caption='Pole powierzchni w różnych układach współrzędnych', label='tab:area'))


\begin{table}[!ht]
\caption{Współrzędne punktów w układzie PL-2000}
\label{tab:2000}
\begin{tabular}{|c|c|c|}
\toprule
nr & x & y \\
\midrule
1 & 5516490.727 & 5957660.601 \\
2 & 5516349.663 & 5997657.405 \\
3 & 5616347.760 & 5998010.911 \\
4 & 5617351.461 & 5958019.885 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[!ht]
\caption{Współrzędne punktów w układzie PL-1992}
\label{tab:1992}
\begin{tabular}{|c|c|c|}
\toprule
nr & x & y \\
\midrule
1 & 252846.028 & 660446.270 \\
2 & 254962.295 & 700391.989 \\
3 & 354799.469 & 695092.080 \\
4 & 353546.705 & 655129.270 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[!ht]
\caption{Współrzędne punktów w układzie UTM}
\label{tab:utm}
\begin{tabular}{|c|c|c|}
\toprule
nr & x & y \\
\midrule
1 & 5955736.129 & 516485.400 \\
2 & 5995720.012 & 516344.382 \\
3 & 5996073.405 & 616310.177 \\
4 & 5956095.297 & 617313.554 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[!ht]
\caption{Współrzędne punktów w układzie LAEA}
\label{tab:la